In [149]:
import os
import scipy
import pandas as pd
import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

In [100]:
# !pip install pandas

In [5]:
mdata = scipy.io.loadmat(os.path.join("data", "mitchell", f"data-science-P{1}.mat"))

In [370]:
#TODO load all participants

### Create voxels-nouns matrix (21764 x 60)

In [284]:
arrays = {}
for item in mdata['info'][0]:
    word = item[2][0]
    if word in arrays:
        arrays[word].append(mdata['data'][np.where(mdata['info'][0] == item)][0][0][0])
    else:
        arrays[word] = [mdata['data'][np.where(mdata['info'][0] == item)][0][0][0]]

In [285]:
averaged_arrays = {}
for word, array in arrays.items():
    averaged_arrays[word] = np.mean(np.stack(arrays[word]), axis=0)

In [367]:
df_voxel_noun = pd.DataFrame.from_dict(averaged_arrays)

In [369]:
df_voxel_noun.head()

,refrigerator,barn,bell,key,window,screwdriver,church,desk,cow,knife,...,door,leg,carrot,watch,table,chimney,bed,fly,dog,telephone
0,-0.119733,-0.199190,0.230340,-0.139996,0.022133,-0.112634,0.353272,-0.042343,-0.189063,-0.072357,...,0.276345,-0.140542,0.094865,-0.540518,-0.051653,-0.213026,0.045836,-0.050042,-0.281979,-0.194655
1,0.249838,-0.189012,0.154014,-0.064165,0.002591,0.137915,0.358341,0.143465,-0.304287,0.051566,...,0.252659,-0.099127,0.151010,-0.521764,-0.219170,-0.097750,0.127190,-0.162893,-0.130701,0.070468
2,0.622086,-0.208923,0.283291,0.100460,-0.197094,0.070147,0.186019,0.318936,-0.328637,0.028104,...,0.037496,-0.326009,0.203993,-0.307184,-0.072724,0.283097,0.055711,-0.173570,-0.028404,0.307875
3,0.042476,-0.221981,0.303526,0.301961,-0.179251,-0.358278,-0.209994,-0.044467,-0.310875,-0.081622,...,0.058031,-0.527249,0.069530,-0.235786,0.005480,0.153367,-0.044356,-0.054538,-0.023114,0.301743
4,0.008095,-0.751286,-0.086668,-0.120380,-0.293255,-0.342001,-0.292562,-0.080478,-0.086134,-0.020103,...,-0.185362,-0.575305,0.118295,-0.653277,-0.558018,0.036400,-0.057903,0.331068,-0.040789,0.281461


### Create sem_featuers-nouns matrix (25 x 60)

In [341]:
semantic_features = {}
with open(raw_file, 'r') as f:
    lines = f.readlines()
    word = None
    for line in lines:
        if len(line) >= 5:
            if "Features for" in line:
                if word and len(semantic_features[word]['features']) < 25: del semantic_features[word] 
                word = line.split("<a name=\"")[1].split("\"")[0]
                semantic_features[word] = { "features": [], "values": []}
            elif word:
                feature_name = line.split()[0]
                val = float(line.split("(")[1].split(")")[0])
                semantic_features[word]["features"].append(feature_name)
                semantic_features[word]["values"].append(val)

In [350]:
df_feature_noun = pd.DataFrame()
for noun, info in semantic_features.items():
    temp_df = pd.DataFrame({
        'feature': info['features'],
        'value': info['values'],
        'noun': noun})
    df_feature_noun = pd.concat([df_feature_noun, temp_df], ignore_index=True)

df_feature_noun = df_feature_noun.pivot(columns='noun', values='value', index='feature')

In [361]:
df_feature_noun.head()

noun,airplane,ant,apartment,arch,arm,barn,bear,bed,bee,beetle,...,shirt,skirt,spoon,table,telephone,tomato,train,truck,watch,window
feature,,,,,,,,,,,,,,,,,,,,,
approach,0.048,0.023,0.009,0.060,0.019,0.009,0.041,0.038,0.175,0.000,...,0.002,0.000,0.010,0.015,0.008,0.000,0.039,0.015,0.003,0.002
break,0.028,0.017,0.064,0.047,0.432,0.008,0.069,0.086,0.302,0.014,...,0.045,0.734,0.085,0.012,0.029,0.010,0.039,0.029,0.013,0.008
clean,0.019,0.132,0.238,0.185,0.021,0.054,0.024,0.192,0.040,0.063,...,0.095,0.016,0.139,0.011,0.006,0.009,0.004,0.037,0.013,0.003
drive,0.023,0.010,0.070,0.075,0.071,0.037,0.085,0.074,0.022,0.702,...,0.011,0.001,0.037,0.009,0.030,0.011,0.489,0.521,0.098,0.007
eat,0.024,0.062,0.012,0.035,0.008,0.006,0.158,0.107,0.092,0.487,...,0.036,0.001,0.741,0.017,0.000,0.260,0.016,0.003,0.049,0.001


### Regression